In [59]:
from reportlab.lib.pagesizes import A4, letter, A5, legal
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# reMarkable tablet page sizes (width x height in mm, converted to points)
REMARKABLE_SIZES = {
    'remarkable1': (157 * mm, 210 * mm),      # reMarkable 1: 10.3" display (157 x 210 mm active area)
    'remarkable2': (157 * mm, 210 * mm),      # reMarkable 2: 10.3" display (157 x 210 mm active area)
    'remarkablemove': (91 * mm, 162 * mm),    # reMarkable Paper Pro Move: 7.3" display (91 x 162 mm active area)
    'remarkablepro': (179 * mm, 239 * mm),    # reMarkable Paper Pro: 11.8" display (179 x 239 mm active area)
    # Aliases for convenience
    'rm1': (157 * mm, 210 * mm),
    'rm2': (157 * mm, 210 * mm),
    'move': (91 * mm, 162 * mm),
    'pro': (179 * mm, 239 * mm),
}

def create_dot_grid_pdf(filename, spacing_mm=5, num_pages=1, page_size=A4, dot_radius=0.5, 
                        page_number_position='lower-right', left_margin_mm=10, right_margin_mm=10,
                        top_margin_mm=10, bottom_margin_mm=10, include_title_page=True, 
                        include_toc=True, toc_line_spacing_mm=10, page_pattern='dots'):
    """
    Create a PDF with a dot grid pattern.
    
    Parameters:
    -----------
    filename : str
        Output PDF filename
    spacing_mm : float
        Spacing between dots in millimeters (default: 5mm)
    num_pages : int
        Number of pages to generate (default: 1)
    page_size : str or tuple
        Page size as either:
        - String: 'remarkable1', 'remarkable2', 'remarkablemove', 'remarkablepro' (or 'rm1', 'rm2', 'move', 'pro')
        - Predefined: A4, letter, A5, legal
        - Custom tuple: (width, height) in points
    dot_radius : float
        Radius of each dot in points (default: 0.5)
    page_number_position : str or None
        Position of page number: 'upper-right', 'upper-middle', 'lower-right', 'lower-middle', 'lower-left', or None to skip page numbering
        (default: 'lower-right')
    left_margin_mm : float
        Left margin in millimeters (default: 10mm)
    right_margin_mm : float
        Right margin in millimeters (default: 10mm)
    top_margin_mm : float
        Top margin in millimeters (default: 10mm)
    bottom_margin_mm : float
        Bottom margin in millimeters (default: 10mm)
    include_title_page : bool
        Whether to include a title page template with blank lines for handwriting (default: True)
    include_toc : bool
        Whether to include a table of contents with hyperlinks to each page (default: True)
    toc_line_spacing_mm : float
        Line spacing for table of contents entries in millimeters (default: 10mm)
    page_pattern : str
        Pattern to draw on each page: 'dots', 'lines', 'grid', or 'blank' (default: 'dots')
    """
    # FIRST: Save original page size string before any conversions
    original_page_size_str = page_size if isinstance(page_size, str) else None
    
    # Validate page number position
    valid_positions = ['upper-right', 'upper-middle', 'lower-right', 'lower-middle', 'lower-left', None]
    if page_number_position not in valid_positions:
        raise ValueError(f"Invalid page_number_position: {page_number_position}. Must be one of {valid_positions}")
    
    # Validate page pattern
    valid_patterns = ['dots', 'lines', 'grid', 'blank']
    if page_pattern not in valid_patterns:
        raise ValueError(f"Invalid page_pattern: {page_pattern}. Must be one of {valid_patterns}")
    
    # Handle string page size input for reMarkable devices
    if isinstance(page_size, str):
        page_size_lower = page_size.lower()
        if page_size_lower in REMARKABLE_SIZES:
            page_size = REMARKABLE_SIZES[page_size_lower]
        else:
            raise ValueError(f"Unknown page size: {page_size}. Use 'remarkable1', 'remarkable2', 'remarkablemove', 'remarkablepro', or a tuple (width, height).")
    
    # Try to register Arial font, fallback to Helvetica if not available
    font_name = "Helvetica"
    try:
        # Try common Arial font paths for different operating systems
        import os
        arial_paths = [
            "C:\\Windows\\Fonts\\arial.ttf",  # Windows
            "/System/Library/Fonts/Supplemental/Arial.ttf",  # macOS
            "/usr/share/fonts/truetype/msttcorefonts/arial.ttf",  # Linux
            "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf"  # Linux alternative
        ]
        
        for path in arial_paths:
            if os.path.exists(path):
                pdfmetrics.registerFont(TTFont('Arial', path))
                font_name = "Arial"
                break
    except:
        pass  # Use Helvetica if Arial registration fails
    
    # Create PDF canvas
    c = canvas.Canvas(filename, pagesize=page_size)
    
    # Get page dimensions
    page_width, page_height = page_size
    
    # Convert spacing from mm to points
    spacing = spacing_mm * mm
    
    # Convert margins from mm to points
    left_margin = left_margin_mm * mm
    right_margin = right_margin_mm * mm
    top_margin = top_margin_mm * mm
    bottom_margin = bottom_margin_mm * mm
    
    # Track actual page number starting from 1
    actual_page_num = 1
    
    # Create title page template if requested
    if include_title_page:
        # Bookmark the title page for linking
        c.bookmarkPage("title_page")
        
        # Also bookmark by actual page number
        c.bookmarkPage(f"page_num_{actual_page_num}")
        
        # Use same margins as dot grid pages
        y_position = page_height - top_margin - 20 - (20 * mm)  # Moved down by 2cm
        line_length = page_width - left_margin - right_margin
        label_x = left_margin
        
        # Set thinner, lighter lines for title page
        c.setLineWidth(0.5)
        c.setStrokeColorRGB(0.7, 0.7, 0.7)
        
        # Set grey color for labels
        c.setFillColorRGB(0.5, 0.5, 0.5)
        c.setFont(font_name, 9)  # Smaller font for title page labels
        
        # Title section - line first, then label beneath (matching other lines)
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 12
        c.drawString(label_x, y_position, "Title")
        y_position -= 28
        
        # Owner section - line first, then label beneath
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 12
        c.drawString(label_x, y_position, "Owner")
        y_position -= 28
        
        # Purpose section (multiple lines) - lines first, then label beneath last line
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 20
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 20
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 12
        c.drawString(label_x, y_position, "Purpose")
        y_position -= 38
        
        # Date Started section - line first, then label beneath
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 12
        c.drawString(label_x, y_position, "Date Started")
        y_position -= 28
        
        # Date Finished section - line first, then label beneath
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 12
        c.drawString(label_x, y_position, "Date Finished")
        
        # Add template description at bottom of page
        # Generate device/size name from original page size string
        if original_page_size_str:
            size_display = {
                'remarkable1': 'reMarkable 1',
                'rm1': 'reMarkable 1',
                'remarkable2': 'reMarkable 2',
                'rm2': 'reMarkable 2',
                'remarkablemove': 'reMarkable Move',
                'move': 'reMarkable Move',
                'remarkablepro': 'reMarkable Paper Pro',
                'pro': 'reMarkable Paper Pro',
                'a4': 'A4',
                'letter': 'Letter',
                'a5': 'A5',
                'legal': 'Legal'
            }.get(original_page_size_str.lower(), original_page_size_str)
        else:
            size_display = 'Custom Size'
        
        template_text = f"Linked Workbook for {size_display}"
        
        # Use italic font and smaller size
        # Helvetica-Oblique is built-in to reportlab, use that for italics
        italic_font = "Helvetica-Oblique"
        c.setFont(italic_font, 7)
        
        c.setFillColorRGB(0.6, 0.6, 0.6)  # Light gray
        
        # Left-justify the text at bottom of page
        text_x = left_margin
        text_y = 10 * mm  # 1cm from bottom
        
        c.drawString(text_x, text_y, template_text)
        
        # Reset to black color and default line width
        c.setFillColorRGB(0, 0, 0)
        c.setStrokeColorRGB(0, 0, 0)
        c.setLineWidth(1)
        
        # End title page and start new page for dot grid
        c.showPage()
        actual_page_num += 1
    
    # Create table of contents if requested
    toc_page_map = {}  # Maps dot-grid page number to TOC page key
    if include_toc:
        # Use same margins as dot grid pages
        toc_line_spacing = toc_line_spacing_mm * mm
        
        # All pages get the same spacing from top (20 points for consistency)
        toc_top_spacing = 20
        toc_header_spacing = 20  # Space between header and first line
        
        # Calculate how many TOC entries fit per page
        # First page has title, subsequent pages don't
        first_page_available_height = page_height - top_margin - bottom_margin - (5 * mm) - (10 * mm) - toc_header_spacing  # Account for title position and spacing
        other_page_available_height = page_height - top_margin - bottom_margin - toc_top_spacing
        
        first_page_entries = int(first_page_available_height / toc_line_spacing)
        entries_per_other_page = int(other_page_available_height / toc_line_spacing)
        
        # Calculate total TOC pages needed
        if num_pages <= first_page_entries:
            num_toc_pages = 1
        else:
            remaining_entries = num_pages - first_page_entries
            num_toc_pages = 1 + ((remaining_entries + entries_per_other_page - 1) // entries_per_other_page)
        
        # Calculate what the first content page's actual page number will be
        first_content_page_num = actual_page_num + num_toc_pages
        
        entry_idx = 0
        for toc_page_idx in range(num_toc_pages):
            # Save the current page number for linking
            current_toc_page_num = actual_page_num
            
            # Bookmark this TOC page
            toc_page_key = f"toc_page_{toc_page_idx + 1}"
            c.bookmarkPage(toc_page_key)
            
            # Also bookmark by actual page number for Previous/Next navigation
            c.bookmarkPage(f"page_num_{current_toc_page_num}")
            
            # Set thinner, lighter lines for TOC on every page
            c.setLineWidth(0.5)
            c.setStrokeColorRGB(0.7, 0.7, 0.7)
            
            # Title only on first page - left-justified, lowered by additional 1cm
            if toc_page_idx == 0:
                c.setFont(font_name, 12)  # Smaller font for TOC heading
                c.setFillColorRGB(0.6, 0.6, 0.6)  # Same grey as page numbers
                title_text = "Table of Contents"
                title_y_position = page_height - top_margin - (5 * mm) - (10 * mm)  # Additional 10mm down
                c.drawString(left_margin, title_y_position, title_text)
                c.setFillColorRGB(0, 0, 0)  # Reset to black
                y_pos = title_y_position - toc_header_spacing  # Only subtract header spacing, not top spacing
                entries_this_page = first_page_entries
            else:
                y_pos = page_height - top_margin - toc_top_spacing
                entries_this_page = entries_per_other_page
            
            # Draw TOC entries for this page
            for _ in range(entries_this_page):
                if entry_idx >= num_pages:
                    break
                    
                dot_page_num = entry_idx + 1
                # Calculate the actual page number for this content page
                actual_content_page_num = first_content_page_num + entry_idx
                
                # Store which TOC page this entry is on
                toc_page_map[dot_page_num] = toc_page_key
                
                # Draw page number (right-justified) using actual page number
                c.setFont(font_name, 10)
                c.setFillColorRGB(0.6, 0.6, 0.6)  # Light gray for page numbers
                page_num_text = str(actual_content_page_num)
                page_num_x = page_width - right_margin
                c.drawRightString(page_num_x, y_pos + 5, page_num_text)
                c.setFillColorRGB(0, 0, 0)  # Reset to black
                
                # Draw line for writing page description - extends all the way through the number
                line_start = left_margin
                line_end = page_num_x + 2  # Extend slightly past the number
                c.line(line_start, y_pos, line_end, y_pos)
                
                # Create clickable link to the dot-grid page
                link_rect = (line_start, y_pos - 5, page_num_x, y_pos + toc_line_spacing - 5)
                c.linkRect("", f"page_{dot_page_num}", link_rect, relative=0)
                
                y_pos -= toc_line_spacing
                entry_idx += 1
            
            # Navigation links at bottom of TOC page
            nav_y_position = 7 * mm  # Lowered from 1cm to 7mm from bottom
            c.setFont(font_name, 10)
            c.setFillColorRGB(0.6, 0.6, 0.6)  # Same gray as page numbers
            
            # Set thin grey line for underlines
            c.setLineWidth(0.5)
            c.setStrokeColorRGB(0.6, 0.6, 0.6)
            
            # Previous link (lower-left) - only if not first TOC page
            if toc_page_idx > 0:
                prev_text = "Previous"
                prev_x = left_margin
                c.drawString(prev_x, nav_y_position, prev_text)
                
                # Underline the text
                text_width = c.stringWidth(prev_text, font_name, 10)
                c.line(prev_x, nav_y_position - 1, prev_x + text_width, nav_y_position - 1)
                
                # Make it clickable to go to previous page
                link_rect = (prev_x - 2, nav_y_position - 2, prev_x + text_width + 2, nav_y_position + 12)
                c.linkRect("", f"page_num_{current_toc_page_num - 1}", link_rect, relative=0)
            
            # Cover link (center) - if title page exists
            if include_title_page:
                cover_text = "Cover"
                cover_width = c.stringWidth(cover_text, font_name, 10)
                cover_x = (page_width - cover_width) / 2
                c.drawString(cover_x, nav_y_position, cover_text)
                
                # Underline the text
                c.line(cover_x, nav_y_position - 1, cover_x + cover_width, nav_y_position - 1)
                
                # Make it clickable to go back to title page
                link_rect = (cover_x - 2, nav_y_position - 2, cover_x + cover_width + 2, nav_y_position + 12)
                c.linkRect("", "title_page", link_rect, relative=0)
            
            # Next link (lower-right) - only if not last TOC page
            if toc_page_idx < num_toc_pages - 1:
                next_text = "Next"
                next_width = c.stringWidth(next_text, font_name, 10)
                next_x = page_width - right_margin - next_width
                c.drawString(next_x, nav_y_position, next_text)
                
                # Underline the text
                c.line(next_x, nav_y_position - 1, next_x + next_width, nav_y_position - 1)
                
                # Make it clickable to go to next page
                link_rect = (next_x - 2, nav_y_position - 2, next_x + next_width + 2, nav_y_position + 12)
                c.linkRect("", f"page_num_{current_toc_page_num + 1}", link_rect, relative=0)
            
            # Reset colors and line width
            c.setFillColorRGB(0, 0, 0)  # Reset to black
            c.setStrokeColorRGB(0, 0, 0)  # Reset stroke to black
            c.setLineWidth(1)  # Reset line width
            
            # Reset stroke color and line width after drawing lines (redundant but clear)
            c.setStrokeColorRGB(0, 0, 0)
            c.setLineWidth(1)
            
            c.showPage()
            actual_page_num += 1
    
    # Generate the specified number of pages
    for page in range(num_pages):
        dot_page_num = page + 1
        
        # Bookmark this page for TOC links
        c.bookmarkPage(f"page_{dot_page_num}")
        
        # Also bookmark by actual page number for navigation
        c.bookmarkPage(f"page_num_{actual_page_num}")
        
        # Calculate starting positions
        start_x = left_margin
        start_y = bottom_margin
        
        # Draw pattern based on page_pattern parameter
        if page_pattern == 'dots':
            # Draw dot grid
            x = start_x
            while x <= page_width - right_margin:
                y = start_y
                while y <= page_height - top_margin:
                    c.circle(x, y, dot_radius, stroke=0, fill=1)
                    y += spacing
                x += spacing
        
        elif page_pattern == 'lines':
            # Set thinner, lighter lines
            c.setLineWidth(0.5)
            c.setStrokeColorRGB(0.7, 0.7, 0.7)
            
            # Draw horizontal lines
            y = start_y
            while y <= page_height - top_margin:
                c.line(start_x, y, page_width - right_margin, y)
                y += spacing
            
            # Reset stroke color and line width
            c.setStrokeColorRGB(0, 0, 0)
            c.setLineWidth(1)
        
        elif page_pattern == 'grid':
            # Set thinner, lighter lines
            c.setLineWidth(0.5)
            c.setStrokeColorRGB(0.7, 0.7, 0.7)
            
            # Calculate how many complete squares fit
            available_width = page_width - left_margin - right_margin
            available_height = page_height - top_margin - bottom_margin
            
            num_squares_horizontal = int(available_width / spacing)
            num_squares_vertical = int(available_height / spacing)
            
            # Calculate actual grid dimensions (only complete squares)
            grid_width = num_squares_horizontal * spacing
            grid_height = num_squares_vertical * spacing
            
            # Center the grid within the margins
            grid_start_x = left_margin + (available_width - grid_width) / 2
            grid_start_y = bottom_margin + (available_height - grid_height) / 2
            
            # Draw horizontal lines (num_squares_vertical + 1 lines)
            for i in range(num_squares_vertical + 1):
                y = grid_start_y + (i * spacing)
                c.line(grid_start_x, y, grid_start_x + grid_width, y)
            
            # Draw vertical lines (num_squares_horizontal + 1 lines)
            for i in range(num_squares_horizontal + 1):
                x = grid_start_x + (i * spacing)
                c.line(x, grid_start_y, x, grid_start_y + grid_height)
            
            # Reset stroke color and line width
            c.setStrokeColorRGB(0, 0, 0)
            c.setLineWidth(1)
        
        # If page_pattern == 'blank', don't draw anything
        
        # Add page number in the specified position (if requested)
        if page_number_position is not None:
            c.setFont(font_name, 14)  # Larger font for page numbers
            c.setFillColorRGB(0.6, 0.6, 0.6)  # Light gray for page numbers
            page_num_text = str(actual_page_num)
            text_width = c.stringWidth(page_num_text, font_name, 14)
            
            # Position page number 1cm (10mm) from bottom
            page_num_bottom = 10 * mm
            
            # For grid pattern, calculate grid edge position to match spacing
            if page_pattern == 'grid' and page_number_position == 'lower-right':
                # Calculate where the grid actually ends
                available_width = page_width - left_margin - right_margin
                available_height = page_height - top_margin - bottom_margin
                num_squares_horizontal = int(available_width / spacing)
                grid_width = num_squares_horizontal * spacing
                grid_start_x = left_margin + (available_width - grid_width) / 2
                grid_end_x = grid_start_x + grid_width
                
                # Position page number with same distance from grid edge as from bottom (about 3mm)
                x_pos = grid_end_x - text_width - (3 * mm)
                y_pos = page_num_bottom
            # Calculate position based on page_number_position parameter for other cases
            elif page_number_position == 'lower-left':
                x_pos = left_margin
                y_pos = page_num_bottom
            elif page_number_position == 'lower-right':
                x_pos = page_width - right_margin - text_width
                y_pos = page_num_bottom
            elif page_number_position == 'lower-middle':
                x_pos = (page_width - text_width) / 2
                y_pos = page_num_bottom
            elif page_number_position == 'upper-right':
                x_pos = page_width - right_margin - text_width
                y_pos = page_height - top_margin / 2 - 10
            elif page_number_position == 'upper-middle':
                x_pos = (page_width - text_width) / 2
                y_pos = page_height - top_margin / 2 - 10
            
            c.drawString(x_pos, y_pos, page_num_text)
            
            # Underline the page number with thin grey line
            c.setLineWidth(0.5)
            c.setStrokeColorRGB(0.6, 0.6, 0.6)
            c.line(x_pos, y_pos - 1, x_pos + text_width, y_pos - 1)
            
            c.setFillColorRGB(0, 0, 0)  # Reset to black
            c.setStrokeColorRGB(0, 0, 0)  # Reset stroke to black
            c.setLineWidth(1)  # Reset line width
            
            # Make page number clickable to return to TOC if TOC is included
            if include_toc and dot_page_num in toc_page_map:
                link_rect = (x_pos - 2, y_pos - 2, x_pos + text_width + 2, y_pos + 14)
                c.linkRect("", toc_page_map[dot_page_num], link_rect, relative=0)
        
        # Create a new page if not the last one
        if page < num_pages - 1:
            c.showPage()
            actual_page_num += 1
    
    # Save the PDF
    c.save()
    
    # Calculate total pages
    total_pages = num_pages
    if include_title_page:
        total_pages += 1
    if include_toc:
        # Calculate TOC pages (matching the logic in the TOC creation section)
        toc_top_spacing = 20
        toc_header_spacing = 20
        first_page_available_height = page_size[1] - top_margin_mm * mm - bottom_margin_mm * mm - (5 * mm) - (10 * mm) - toc_header_spacing
        other_page_available_height = page_size[1] - top_margin_mm * mm - bottom_margin_mm * mm - toc_top_spacing
        toc_spacing_points = toc_line_spacing_mm * mm
        
        first_page_entries = int(first_page_available_height / toc_spacing_points)
        entries_per_other_page = int(other_page_available_height / toc_spacing_points)
        
        if num_pages <= first_page_entries:
            num_toc_pages = 1
        else:
            remaining_entries = num_pages - first_page_entries
            num_toc_pages = 1 + ((remaining_entries + entries_per_other_page - 1) // entries_per_other_page)
        
        total_pages += num_toc_pages
    
    print(f"PDF created: {filename}")
    print(f"  - Content pages: {num_pages}")
    if include_title_page:
        print(f"  - Title page: Yes")
    if include_toc:
        print(f"  - Table of Contents: Yes ({num_toc_pages} page(s))")
    print(f"  - Total pages in PDF: {total_pages}")
    print(f"  - Page pattern: {page_pattern}")
    if page_pattern in ['dots', 'lines', 'grid']:
        print(f"  - Pattern spacing: {spacing_mm}mm")
    print(f"  - Page size: {page_size[0]/mm:.1f}mm x {page_size[1]/mm:.1f}mm")
    print(f"  - Margins: L={left_margin_mm}mm, R={right_margin_mm}mm, T={top_margin_mm}mm, B={bottom_margin_mm}mm")
    print(f"  - Page number position: {page_number_position if page_number_position else 'None (no page numbers)'}")
    print(f"  - Font used: {font_name}")


# Example usage
if __name__ == "__main__":
    # Customize these parameters:
    DOT_SPACING_MM = 5  # spacing between dots in millimeters
    NUM_PAGES = 256       # number of pages
    DOT_SIZE = 0.5      # radius of dots in points
    PAGE_NUM_POS = 'lower-right'  # 'upper-right', 'upper-middle', 'lower-right', 'lower-middle', 'lower-left'
    INCLUDE_TITLE_PAGE = True  # Set to False to skip the title page template
    INCLUDE_TOC = True  # Set to False to skip the table of contents
    TOC_LINE_SPACING = 8  # Line spacing for TOC entries in millimeters
    
    # Margins in millimeters
    LEFT_MARGIN = 5
    RIGHT_MARGIN = 5
    TOP_MARGIN = 5
    BOTTOM_MARGIN = 5
    
    # Generate all combinations for reMarkable devices
    devices = ['remarkable1', 'remarkable2', 'move', 'pro']
    patterns = ['dots', 'lines', 'grid', 'blank']
    
    # Display names for devices and patterns
    device_names = {
        'remarkable1': 'reMarkable 1',
        'remarkable2': 'reMarkable 2',
        'move': 'reMarkable Move',
        'pro': 'reMarkable Paper Pro'
    }
    
    pattern_names = {
        'dots': 'Dots',
        'lines': 'Lines',
        'grid': 'Grid',
        'blank': 'Blank'
    }
    
    for device in devices:
        for pattern in patterns:
            device_display = device_names[device]
            pattern_display = pattern_names[pattern]
            OUTPUT_FILE = f"{device_display} - {pattern_display}.pdf"
            
            print(f"\n{'='*60}")
            print(f"Generating {OUTPUT_FILE}...")
            print(f"{'='*60}")
            
            # Create the PDF
            create_dot_grid_pdf(
                filename=OUTPUT_FILE,
                spacing_mm=DOT_SPACING_MM,
                num_pages=NUM_PAGES,
                page_size=device,
                dot_radius=DOT_SIZE,
                page_number_position=PAGE_NUM_POS,
                left_margin_mm=LEFT_MARGIN,
                right_margin_mm=RIGHT_MARGIN,
                top_margin_mm=TOP_MARGIN,
                bottom_margin_mm=BOTTOM_MARGIN,
                include_title_page=INCLUDE_TITLE_PAGE,
                include_toc=INCLUDE_TOC,
                toc_line_spacing_mm=TOC_LINE_SPACING,
                page_pattern=pattern
            )
    
    print(f"\n{'='*60}")
    print(f"✓ Successfully generated {len(devices) * len(patterns)} PDF files!")
    print(f"{'='*60}")
    
    # Examples with different configurations:
    # create_dot_grid_pdf("dot_grid_3mm.pdf", spacing_mm=3, num_pages=10, page_pattern='dots')
    # create_dot_grid_pdf("dot_grid_letter.pdf", spacing_mm=5, num_pages=5, page_size=letter)
    # create_dot_grid_pdf("dot_grid_custom.pdf", spacing_mm=7, num_pages=3, page_size=(200*mm, 250*mm))
    
    # reMarkable tablet examples:
    # create_dot_grid_pdf("rm1_notebook.pdf", spacing_mm=5, num_pages=50, page_size="remarkable1")
    # create_dot_grid_pdf("rm2_notebook.pdf", spacing_mm=5, num_pages=50, page_size="remarkable2")
    # create_dot_grid_pdf("rm_move.pdf", spacing_mm=5, num_pages=50, page_size="move")
    # create_dot_grid_pdf("rm_pro.pdf", spacing_mm=5, num_pages=50, page_size="pro")
    
    # Page number position examples:
    # create_dot_grid_pdf("upper_right.pdf", num_pages=10, page_number_position="upper-right")
    # create_dot_grid_pdf("upper_middle.pdf", num_pages=10, page_number_position="upper-middle")
    # create_dot_grid_pdf("lower_middle.pdf", num_pages=10, page_number_position="lower-middle")
    # create_dot_grid_pdf("lower_left.pdf", num_pages=10, page_number_position="lower-left")
    # create_dot_grid_pdf("no_numbers.pdf", num_pages=10, page_number_position=None)  # Skip page numbers
    
    # Skip the title page if desired:
    # create_dot_grid_pdf("no_title.pdf", num_pages=50, include_title_page=False)
    
    # Skip the table of contents:
    # create_dot_grid_pdf("no_toc.pdf", num_pages=50, include_toc=False)
    
    # Adjust TOC line spacing for more/fewer entries per page:
    # create_dot_grid_pdf("compact_toc.pdf", num_pages=100, toc_line_spacing_mm=8)
    # create_dot_grid_pdf("spacious_toc.pdf", num_pages=100, toc_line_spacing_mm=15)
    
    # Different page patterns:
    # create_dot_grid_pdf("lined.pdf", num_pages=100, page_pattern='lines', spacing_mm=7)
    # create_dot_grid_pdf("grid.pdf", num_pages=100, page_pattern='grid', spacing_mm=5)
    # create_dot_grid_pdf("blank.pdf", num_pages=100, page_pattern='blank')
    
    # Create a notebook with both title page and TOC (default):
    # create_dot_grid_pdf("complete_notebook.pdf", num_pages=100, page_size="remarkable2")
    
    # Custom margins example:
    # create_dot_grid_pdf("wide_margins.pdf", num_pages=10, 
    #                     left_margin_mm=20, right_margin_mm=20,
    #                     top_margin_mm=25, bottom_margin_mm=15)


Generating reMarkable 1 - Dots.pdf...
PDF created: reMarkable 1 - Dots.pdf
  - Content pages: 256
  - Title page: Yes
  - Table of Contents: Yes (11 page(s))
  - Total pages in PDF: 268
  - Page pattern: dots
  - Pattern spacing: 5mm
  - Page size: 157.0mm x 210.0mm
  - Margins: L=5mm, R=5mm, T=5mm, B=5mm
  - Page number position: lower-right
  - Font used: Arial

Generating reMarkable 1 - Lines.pdf...
PDF created: reMarkable 1 - Lines.pdf
  - Content pages: 256
  - Title page: Yes
  - Table of Contents: Yes (11 page(s))
  - Total pages in PDF: 268
  - Page pattern: lines
  - Pattern spacing: 5mm
  - Page size: 157.0mm x 210.0mm
  - Margins: L=5mm, R=5mm, T=5mm, B=5mm
  - Page number position: lower-right
  - Font used: Arial

Generating reMarkable 1 - Grid.pdf...
PDF created: reMarkable 1 - Grid.pdf
  - Content pages: 256
  - Title page: Yes
  - Table of Contents: Yes (11 page(s))
  - Total pages in PDF: 268
  - Page pattern: grid
  - Pattern spacing: 5mm
  - Page size: 157.0mm x 210

In [ ]:
change the output file name to include the device name and the grip

In [ ]:
from reportlab.lib.pagesizes import A4, letter, A5, legal
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# reMarkable tablet page sizes (width x height in mm, converted to points)
REMARKABLE_SIZES = {
    'remarkable1': (157 * mm, 210 * mm),      # reMarkable 1: 10.3" display (157 x 210 mm active area)
    'remarkable2': (157 * mm, 210 * mm),      # reMarkable 2: 10.3" display (157 x 210 mm active area)
    'remarkablemove': (91 * mm, 162 * mm),    # reMarkable Paper Pro Move: 7.3" display (91 x 162 mm active area)
    'remarkablepro': (179 * mm, 239 * mm),    # reMarkable Paper Pro: 11.8" display (179 x 239 mm active area)
    # Aliases for convenience
    'rm1': (157 * mm, 210 * mm),
    'rm2': (157 * mm, 210 * mm),
    'move': (91 * mm, 162 * mm),
    'pro': (179 * mm, 239 * mm),
    # Onyx Boox devices (portrait orientation: width x height in mm)
    'booxnoteair': (157 * mm, 209 * mm),      # Onyx Boox Note Air: 10.3" display (1872×1404)
    'booxmaxlumi': (203 * mm, 270 * mm),      # Onyx Boox Max Lumi: 13.3" display (2200×1650)
    'booxnoteair3c': (157 * mm, 209 * mm),    # Onyx Boox Note Air 3C: 10.3" display (1872×1404)
    'booxnoteair3': (157 * mm, 209 * mm),     # Onyx Boox Note Air 3: 10.3" display (1872×1404)
    'booxnoteair4c': (157 * mm, 209 * mm),    # Onyx Boox Note Air 4C: 10.3" display (2480×1860)
    'booxtabminic': (119 * mm, 158 * mm),     # Onyx Boox Tab Mini C: 7.8" display (1872×1404)
    'booxnotemax': (203 * mm, 270 * mm),      # Onyx Boox Note Max: 13.3" display (3200×2400)
    'booxgo103': (157 * mm, 209 * mm),        # Onyx Boox Go 10.3: 10.3" display (2480×1860)
}

def create_dot_grid_pdf(filename, spacing_mm=5, num_pages=1, page_size=A4, dot_radius=0.5, 
                        page_number_position='lower-right', left_margin_mm=10, right_margin_mm=10,
                        top_margin_mm=10, bottom_margin_mm=10, include_title_page=True, 
                        include_toc=True, toc_line_spacing_mm=10, page_pattern='dots'):
    """
    Create a PDF with a dot grid pattern.
    
    Parameters:
    -----------
    filename : str
        Output PDF filename
    spacing_mm : float
        Spacing between dots in millimeters (default: 5mm)
    num_pages : int
        Number of pages to generate (default: 1)
    page_size : str or tuple
        Page size as either:
        - String: 'remarkable1', 'remarkable2', 'remarkablemove', 'remarkablepro' (or 'rm1', 'rm2', 'move', 'pro')
        - Predefined: A4, letter, A5, legal
        - Custom tuple: (width, height) in points
    dot_radius : float
        Radius of each dot in points (default: 0.5)
    page_number_position : str or None
        Position of page number: 'upper-right', 'upper-middle', 'lower-right', 'lower-middle', 'lower-left', or None to skip page numbering
        (default: 'lower-right')
    left_margin_mm : float
        Left margin in millimeters (default: 10mm)
    right_margin_mm : float
        Right margin in millimeters (default: 10mm)
    top_margin_mm : float
        Top margin in millimeters (default: 10mm)
    bottom_margin_mm : float
        Bottom margin in millimeters (default: 10mm)
    include_title_page : bool
        Whether to include a title page template with blank lines for handwriting (default: True)
    include_toc : bool
        Whether to include a table of contents with hyperlinks to each page (default: True)
    toc_line_spacing_mm : float
        Line spacing for table of contents entries in millimeters (default: 10mm)
    page_pattern : str
        Pattern to draw on each page: 'dots', 'lines', 'grid', or 'blank' (default: 'dots')
    """
    # FIRST: Save original page size string before any conversions
    original_page_size_str = page_size if isinstance(page_size, str) else None
    
    # Validate page number position
    valid_positions = ['upper-right', 'upper-middle', 'lower-right', 'lower-middle', 'lower-left', None]
    if page_number_position not in valid_positions:
        raise ValueError(f"Invalid page_number_position: {page_number_position}. Must be one of {valid_positions}")
    
    # Validate page pattern
    valid_patterns = ['dots', 'lines', 'grid', 'blank']
    if page_pattern not in valid_patterns:
        raise ValueError(f"Invalid page_pattern: {page_pattern}. Must be one of {valid_patterns}")
    
    # Handle string page size input for reMarkable devices
    if isinstance(page_size, str):
        page_size_lower = page_size.lower()
        if page_size_lower in REMARKABLE_SIZES:
            page_size = REMARKABLE_SIZES[page_size_lower]
        else:
            raise ValueError(f"Unknown page size: {page_size}. Use a reMarkable device name (remarkable1, remarkable2, remarkablemove, remarkablepro), Onyx Boox device name (booxnoteair, booxmaxlumi, booxnoteair3c, booxtabminic, booxnotemax, booxgo103), or a tuple (width, height).")
    
    # Try to register Arial font, fallback to Helvetica if not available
    font_name = "Helvetica"
    try:
        # Try common Arial font paths for different operating systems
        import os
        arial_paths = [
            "C:\\Windows\\Fonts\\arial.ttf",  # Windows
            "/System/Library/Fonts/Supplemental/Arial.ttf",  # macOS
            "/usr/share/fonts/truetype/msttcorefonts/arial.ttf",  # Linux
            "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf"  # Linux alternative
        ]
        
        for path in arial_paths:
            if os.path.exists(path):
                pdfmetrics.registerFont(TTFont('Arial', path))
                font_name = "Arial"
                break
    except:
        pass  # Use Helvetica if Arial registration fails
    
    # Try to register EB Garamond font for serif text
    serif_font_name = "Helvetica"  # Default fallback
    serif_font_embedded = False
    try:
        import os
        garamond_paths = [
            "C:\\Windows\\Fonts\\EBGaramond-Regular.ttf",  # Windows
            "/Library/Fonts/EBGaramond-Regular.ttf",  # macOS system
            "/System/Library/Fonts/Supplemental/EBGaramond-Regular.ttf",  # macOS supplemental
            "~/Library/Fonts/EBGaramond-Regular.ttf",  # macOS user
            "~/Library/Fonts/EBGaramond-VariableFont_wght.ttf",  # macOS user variable font
            "/usr/share/fonts/truetype/eb-garamond/EBGaramond-Regular.ttf",  # Linux
            "/usr/share/fonts/truetype/ebgaramond/EBGaramond-Regular.ttf",  # Linux alternative
            "/usr/share/fonts/opentype/eb-garamond/EBGaramond-Regular.otf",  # Linux OTF
        ]
        
        for path in garamond_paths:
            expanded_path = os.path.expanduser(path)
            if os.path.exists(expanded_path):
                # TTFont automatically embeds the font in the PDF
                pdfmetrics.registerFont(TTFont('EBGaramond', expanded_path))
                serif_font_name = "EBGaramond"
                serif_font_embedded = True
                break
    except:
        pass  # Use Helvetica if EB Garamond registration fails
    
    # Create PDF canvas
    c = canvas.Canvas(filename, pagesize=page_size)
    
    # Get page dimensions
    page_width, page_height = page_size
    
    # Convert spacing from mm to points
    spacing = spacing_mm * mm
    
    # Convert margins from mm to points
    left_margin = left_margin_mm * mm
    right_margin = right_margin_mm * mm
    top_margin = top_margin_mm * mm
    bottom_margin = bottom_margin_mm * mm
    
    # Track actual page number starting from 1
    actual_page_num = 1
    
    # Create title page template if requested
    if include_title_page:
        # Bookmark the title page for linking
        c.bookmarkPage("title_page")
        
        # Also bookmark by actual page number
        c.bookmarkPage(f"page_num_{actual_page_num}")
        
        # Add "Linked Workbook" header at the top
        header_y = page_height - top_margin - (20 * mm)  # Raised by 5mm from previous position
        c.setFont(serif_font_name, 16)  # Same size as TOC header
        c.setFillColorRGB(0.3, 0.3, 0.3)  # Dark gray
        header_text = "Linked Workbook"
        # Left-justify the header
        header_x = left_margin
        c.drawString(header_x, header_y, header_text)
        
        # Reset color
        c.setFillColorRGB(0, 0, 0)
        
        # Use same margins as dot grid pages, lowered further
        y_position = page_height - top_margin - 20 - (30 * mm)  # Lowered significantly
        line_length = page_width - left_margin - right_margin
        label_x = left_margin
        
        # Set thinner, lighter lines for title page
        c.setLineWidth(0.5)
        c.setStrokeColorRGB(0.7, 0.7, 0.7)
        
        # Set grey color for labels
        c.setFillColorRGB(0.3, 0.3, 0.3)
        c.setFont(font_name, 7)  # Smaller font for title page labels
        
        # Title section - line first, then label beneath (matching other lines)
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 12
        c.drawString(label_x, y_position, "Title")
        y_position -= 28
        
        # Owner section - line first, then label beneath
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 12
        c.drawString(label_x, y_position, "Owner")
        y_position -= 28
        
        # Description section (multiple lines) - lines first, then label beneath last line
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 20
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 20
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 12
        c.drawString(label_x, y_position, "Description")
        y_position -= 38
        
        # Date From section - line first, then label beneath
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 12
        c.drawString(label_x, y_position, "Date From")
        y_position -= 28
        
        # Date To section - line first, then label beneath
        c.line(label_x, y_position, label_x + line_length, y_position)
        y_position -= 12
        c.drawString(label_x, y_position, "Date To")
        
        # Add template description at bottom of page
        # Generate device/size name from original page size string
        if original_page_size_str:
            size_display = {
                'remarkable1': 'reMarkable 1',
                'rm1': 'reMarkable 1',
                'remarkable2': 'reMarkable 2',
                'rm2': 'reMarkable 2',
                'remarkablemove': 'reMarkable Paper Pro Move',
                'move': 'reMarkable Paper Pro Move',
                'remarkablepro': 'reMarkable Paper Pro',
                'pro': 'reMarkable Paper Pro',
                'booxnoteair': 'Onyx Boox Note Air',
                'booxmaxlumi': 'Onyx Boox Max Lumi',
                'booxnoteair3c': 'Onyx Boox Note Air 3C',
                'booxnoteair3': 'Onyx Boox Note Air 3',
                'booxnoteair4c': 'Onyx Boox Note Air 4C',
                'booxtabminic': 'Onyx Boox Tab Mini C',
                'booxnotemax': 'Onyx Boox Note Max',
                'booxgo103': 'Onyx Boox Go 10.3',
                'a4': 'A4',
                'letter': 'Letter',
                'a5': 'A5',
                'legal': 'Legal'
            }.get(original_page_size_str.lower(), original_page_size_str)
        else:
            size_display = 'Custom Size'
        
        template_text = f"For {size_display}"
        
        # Use italic font and smaller size
        # Helvetica-Oblique is built-in to reportlab, use that for italics
        italic_font = "Helvetica-Oblique"
        c.setFont(italic_font, 6)  # Smaller font size
        
        c.setFillColorRGB(0.4, 0.4, 0.4)  # Darker gray
        
        # Left-justify the text at bottom of page
        text_x = left_margin
        text_y = 10 * mm  # 1cm from bottom
        
        c.drawString(text_x, text_y, template_text)
        
        # Reset to black color and default line width
        c.setFillColorRGB(0, 0, 0)
        c.setStrokeColorRGB(0, 0, 0)
        c.setLineWidth(1)
        
        # End title page and start new page for dot grid
        c.showPage()
        actual_page_num += 1
    
    # Create table of contents if requested
    toc_page_map = {}  # Maps dot-grid page number to TOC page key
    if include_toc:
        # Use same margins as dot grid pages
        toc_line_spacing = toc_line_spacing_mm * mm
        
        # All pages get the same spacing from top (20 points for consistency) + 1cm lower
        toc_top_spacing = 20 + (10 * mm)  # Added 10mm (1cm) to lower the content
        toc_header_spacing = 20  # Space between header and first line
        
        # Calculate how many TOC entries fit per page
        # First page has title, subsequent pages don't
        first_page_available_height = page_height - top_margin - bottom_margin - (5 * mm) - (20 * mm) - toc_header_spacing  # Lowered by additional 10mm (1cm)
        other_page_available_height = page_height - top_margin - bottom_margin - toc_top_spacing
        
        first_page_entries = int(first_page_available_height / toc_line_spacing)
        entries_per_other_page = int(other_page_available_height / toc_line_spacing)
        
        # Calculate total TOC pages needed
        if num_pages <= first_page_entries:
            num_toc_pages = 1
        else:
            remaining_entries = num_pages - first_page_entries
            num_toc_pages = 1 + ((remaining_entries + entries_per_other_page - 1) // entries_per_other_page)
        
        # Calculate what the first content page's actual page number will be
        first_content_page_num = actual_page_num + num_toc_pages
        
        entry_idx = 0
        for toc_page_idx in range(num_toc_pages):
            # Save the current page number for linking
            current_toc_page_num = actual_page_num
            
            # Bookmark this TOC page
            toc_page_key = f"toc_page_{toc_page_idx + 1}"
            c.bookmarkPage(toc_page_key)
            
            # Also bookmark by actual page number for Previous/Next navigation
            c.bookmarkPage(f"page_num_{current_toc_page_num}")
            
            # Set thinner, lighter lines for TOC on every page
            c.setLineWidth(0.5)
            c.setStrokeColorRGB(0.7, 0.7, 0.7)
            
            # Title only on first page - left-justified
            # Position matches Linked Workbook heading exactly
            if toc_page_idx == 0:
                c.setFont(serif_font_name, 16)  # Use serif font for TOC heading
                c.setFillColorRGB(0.4, 0.4, 0.4)  # Darker grey
                title_text = "Table of Contents"
                title_y_position = page_height - top_margin - (15 * mm)
                c.drawString(left_margin, title_y_position, title_text)
                c.setFillColorRGB(0, 0, 0)  # Reset to black
                y_pos = title_y_position - toc_header_spacing  # Only subtract header spacing, not top spacing
                entries_this_page = first_page_entries
            else:
                y_pos = page_height - top_margin - toc_top_spacing
                entries_this_page = entries_per_other_page
            
            # Draw TOC entries for this page
            for _ in range(entries_this_page):
                if entry_idx >= num_pages:
                    break
                    
                dot_page_num = entry_idx + 1
                # Display page number starting from 1 for content pages
                display_page_num = entry_idx + 1
                
                # Store which TOC page this entry is on
                toc_page_map[dot_page_num] = toc_page_key
                
                # Draw page number (right-justified) using display page number
                c.setFont(font_name, 8)  # Further reduced font size
                c.setFillColorRGB(0, 0, 0)  # Black for page numbers
                page_num_text = str(display_page_num)
                page_num_x = page_width - right_margin + (2 * mm)  # Moved 2mm to the right
                c.drawRightString(page_num_x, y_pos + 5, page_num_text)
                c.setFillColorRGB(0, 0, 0)  # Reset to black
                
                # Draw line for writing page description - extends all the way through the number
                line_start = left_margin
                line_end = page_num_x + 2  # Extend slightly past the number
                c.line(line_start, y_pos, line_end, y_pos)
                
                # Create clickable link to the dot-grid page
                link_rect = (line_start, y_pos - 5, page_num_x, y_pos + toc_line_spacing - 5)
                c.linkRect("", f"page_{dot_page_num}", link_rect, relative=0)
                
                y_pos -= toc_line_spacing
                entry_idx += 1
            
            # Navigation links at bottom of TOC page - raised by 5mm
            nav_y_position = 12 * mm  # Raised from 7mm to 12mm (5mm increase)
            c.setFont(font_name, 8)  # Smaller font for navigation links
            c.setFillColorRGB(0.4, 0.4, 0.4)  # Darker gray
            
            # Previous link (lower-left) - only if not first TOC page
            if toc_page_idx > 0:
                prev_text = "Previous"
                prev_x = left_margin
                c.drawString(prev_x, nav_y_position, prev_text)
                
                # Make it clickable to go to previous page
                text_width = c.stringWidth(prev_text, font_name, 8)
                link_rect = (prev_x - 2, nav_y_position - 2, prev_x + text_width + 2, nav_y_position + 12)
                c.linkRect("", f"page_num_{current_toc_page_num - 1}", link_rect, relative=0)
            
            # Cover link (center) - if title page exists
            if include_title_page:
                cover_text = "Cover"
                cover_width = c.stringWidth(cover_text, font_name, 8)
                cover_x = (page_width - cover_width) / 2
                c.drawString(cover_x, nav_y_position, cover_text)
                
                # Make it clickable to go back to title page
                link_rect = (cover_x - 2, nav_y_position - 2, cover_x + cover_width + 2, nav_y_position + 12)
                c.linkRect("", "title_page", link_rect, relative=0)
            
            # Next link (lower-right) - only if not last TOC page
            if toc_page_idx < num_toc_pages - 1:
                next_text = "Next"
                next_width = c.stringWidth(next_text, font_name, 8)
                next_x = page_width - right_margin - next_width
                c.drawString(next_x, nav_y_position, next_text)
                
                # Make it clickable to go to next page
                link_rect = (next_x - 2, nav_y_position - 2, next_x + next_width + 2, nav_y_position + 12)
                c.linkRect("", f"page_num_{current_toc_page_num + 1}", link_rect, relative=0)
            
            # Reset colors and line width
            c.setFillColorRGB(0, 0, 0)  # Reset to black
            c.setStrokeColorRGB(0, 0, 0)  # Reset stroke to black
            c.setLineWidth(1)  # Reset line width
            
            # Reset stroke color and line width after drawing lines (redundant but clear)
            c.setStrokeColorRGB(0, 0, 0)
            c.setLineWidth(1)
            
            c.showPage()
            actual_page_num += 1
    
    # Generate the specified number of pages
    for page in range(num_pages):
        dot_page_num = page + 1
        display_page_num = page + 1  # Content pages numbered starting from 1
        
        # Bookmark this page for TOC links
        c.bookmarkPage(f"page_{dot_page_num}")
        
        # Also bookmark by actual page number for navigation
        c.bookmarkPage(f"page_num_{actual_page_num}")
        
        # Calculate starting positions
        start_x = left_margin
        start_y = bottom_margin
        
        # Draw pattern based on page_pattern parameter
        if page_pattern == 'dots':
            # Draw dot grid
            x = start_x
            while x <= page_width - right_margin:
                y = start_y
                while y <= page_height - top_margin:
                    c.circle(x, y, dot_radius, stroke=0, fill=1)
                    y += spacing
                x += spacing
        
        elif page_pattern == 'lines':
            # Set thinner, lighter lines
            c.setLineWidth(0.5)
            c.setStrokeColorRGB(0.7, 0.7, 0.7)
            
            # Draw horizontal lines
            y = start_y
            while y <= page_height - top_margin:
                c.line(start_x, y, page_width - right_margin, y)
                y += spacing
            
            # Reset stroke color and line width
            c.setStrokeColorRGB(0, 0, 0)
            c.setLineWidth(1)
        
        elif page_pattern == 'grid':
            # Set thinner, lighter lines
            c.setLineWidth(0.5)
            c.setStrokeColorRGB(0.7, 0.7, 0.7)
            
            # Calculate how many complete squares fit
            available_width = page_width - left_margin - right_margin
            available_height = page_height - top_margin - bottom_margin
            
            num_squares_horizontal = int(available_width / spacing)
            num_squares_vertical = int(available_height / spacing)
            
            # Calculate actual grid dimensions (only complete squares)
            grid_width = num_squares_horizontal * spacing
            grid_height = num_squares_vertical * spacing
            
            # Center the grid within the margins
            grid_start_x = left_margin + (available_width - grid_width) / 2
            grid_start_y = bottom_margin + (available_height - grid_height) / 2
            
            # Draw horizontal lines (num_squares_vertical + 1 lines)
            for i in range(num_squares_vertical + 1):
                y = grid_start_y + (i * spacing)
                c.line(grid_start_x, y, grid_start_x + grid_width, y)
            
            # Draw vertical lines (num_squares_horizontal + 1 lines)
            for i in range(num_squares_horizontal + 1):
                x = grid_start_x + (i * spacing)
                c.line(x, grid_start_y, x, grid_start_y + grid_height)
            
            # Reset stroke color and line width
            c.setStrokeColorRGB(0, 0, 0)
            c.setLineWidth(1)
        
        # If page_pattern == 'blank', don't draw anything
        
        # Add page number in the specified position (if requested)
        if page_number_position is not None:
            c.setFont(font_name, 14)  # Larger font for page numbers
            c.setFillColorRGB(0.55, 0.55, 0.55)  # Lighter gray for page numbers
            page_num_text = str(display_page_num)
            text_width = c.stringWidth(page_num_text, font_name, 14)
            
            # Position page number 1.1cm (11mm) from bottom
            page_num_bottom = 11 * mm
            
            # For grid pattern, calculate grid edge position to match spacing
            if page_pattern == 'grid' and page_number_position == 'lower-right':
                # Calculate where the grid actually ends
                available_width = page_width - left_margin - right_margin
                available_height = page_height - top_margin - bottom_margin
                num_squares_horizontal = int(available_width / spacing)
                grid_width = num_squares_horizontal * spacing
                grid_start_x = left_margin + (available_width - grid_width) / 2
                grid_end_x = grid_start_x + grid_width
                
                # Position page number with same distance from grid edge as from bottom (about 3mm)
                x_pos = grid_end_x - text_width - (3 * mm)
                y_pos = page_num_bottom
            # Calculate position based on page_number_position parameter for other cases
            elif page_number_position == 'lower-left':
                x_pos = left_margin
                y_pos = page_num_bottom
            elif page_number_position == 'lower-right':
                x_pos = page_width - right_margin - text_width
                y_pos = page_num_bottom
            elif page_number_position == 'lower-middle':
                x_pos = (page_width - text_width) / 2
                y_pos = page_num_bottom
            elif page_number_position == 'upper-right':
                x_pos = page_width - right_margin - text_width
                y_pos = page_height - top_margin / 2 - 10
            elif page_number_position == 'upper-middle':
                x_pos = (page_width - text_width) / 2
                y_pos = page_height - top_margin / 2 - 10
            
            c.drawString(x_pos, y_pos, page_num_text)
            
            c.setFillColorRGB(0, 0, 0)  # Reset to black
            c.setStrokeColorRGB(0, 0, 0)  # Reset stroke to black
            c.setLineWidth(1)  # Reset line width
            
            # Make page number clickable to return to TOC if TOC is included
            if include_toc and dot_page_num in toc_page_map:
                link_rect = (x_pos - 2, y_pos - 2, x_pos + text_width + 2, y_pos + 14)
                c.linkRect("", toc_page_map[dot_page_num], link_rect, relative=0)
        
        # Create a new page if not the last one
        if page < num_pages - 1:
            c.showPage()
            actual_page_num += 1
    
    # Save the PDF
    c.save()
    
    # Calculate total pages
    total_pages = num_pages
    if include_title_page:
        total_pages += 1
    if include_toc:
        # Calculate TOC pages (matching the logic in the TOC creation section)
        toc_top_spacing_calc = 20 + (10 * mm)  # Updated to match the new spacing
        toc_header_spacing = 20
        first_page_available_height = page_size[1] - top_margin_mm * mm - bottom_margin_mm * mm - (15 * mm) - toc_header_spacing  # Updated
        other_page_available_height = page_size[1] - top_margin_mm * mm - bottom_margin_mm * mm - toc_top_spacing_calc
        toc_spacing_points = toc_line_spacing_mm * mm
        
        first_page_entries = int(first_page_available_height / toc_spacing_points)
        entries_per_other_page = int(other_page_available_height / toc_spacing_points)
        
        if num_pages <= first_page_entries:
            num_toc_pages = 1
        else:
            remaining_entries = num_pages - first_page_entries
            num_toc_pages = 1 + ((remaining_entries + entries_per_other_page - 1) // entries_per_other_page)
        
        total_pages += num_toc_pages
    
    print(f"PDF created: {filename}")
    print(f"  - Content pages: {num_pages}")
    if include_title_page:
        print(f"  - Title page: Yes")
    if include_toc:
        print(f"  - Table of Contents: Yes ({num_toc_pages} page(s))")
    print(f"  - Total pages in PDF: {total_pages}")
    print(f"  - Page pattern: {page_pattern}")
    if page_pattern in ['dots', 'lines', 'grid']:
        print(f"  - Pattern spacing: {spacing_mm}mm")
    print(f"  - Page size: {page_size[0]/mm:.1f}mm x {page_size[1]/mm:.1f}mm")
    print(f"  - Margins: L={left_margin_mm}mm, R={right_margin_mm}mm, T={top_margin_mm}mm, B={bottom_margin_mm}mm")
    print(f"  - Page number position: {page_number_position if page_number_position else 'None (no page numbers)'}")
    print(f"  - Font used: {font_name}")
    if serif_font_embedded:
        print(f"  - Serif font used: {serif_font_name} (embedded in PDF)")
    else:
        print(f"  - Serif font used: {serif_font_name} (fallback, not embedded)")


# Example usage
if __name__ == "__main__":
    # Customize these parameters:
    DOT_SPACING_MM = 5  # spacing between dots in millimeters
    NUM_PAGES = 256       # number of pages
    DOT_SIZE = 0.5      # radius of dots in points
    PAGE_NUM_POS = 'lower-left'  # 'upper-right', 'upper-middle', 'lower-right', 'lower-middle', 'lower-left'
    INCLUDE_TITLE_PAGE = True  # Set to False to skip the title page template
    INCLUDE_TOC = True  # Set to False to skip the table of contents
    TOC_LINE_SPACING = 8  # Line spacing for TOC entries in millimeters
    
    # Margins in millimeters
    LEFT_MARGIN = 5
    RIGHT_MARGIN = 5
    TOP_MARGIN = 5
    BOTTOM_MARGIN = 5
    
    # Generate all combinations for reMarkable and Onyx Boox devices
    devices = [
        'remarkable1', 'remarkable2', 'move', 'pro',
        'booxnoteair', 'booxmaxlumi', 'booxnoteair3c', 'booxnoteair3', 
        'booxnoteair4c', 'booxtabminic', 'booxnotemax', 'booxgo103'
    ]
    patterns = ['dots', 'lines', 'grid', 'blank']
    
    # Display names for devices and patterns
    device_names = {
        'remarkable1': 'reMarkable 1',
        'remarkable2': 'reMarkable 2',
        'move': 'reMarkable Paper Pro Move',
        'pro': 'reMarkable Paper Pro',
        'booxnoteair': 'Onyx Boox Note Air',
        'booxmaxlumi': 'Onyx Boox Max Lumi',
        'booxnoteair3c': 'Onyx Boox Note Air 3C',
        'booxnoteair3': 'Onyx Boox Note Air 3',
        'booxnoteair4c': 'Onyx Boox Note Air 4C',
        'booxtabminic': 'Onyx Boox Tab Mini C',
        'booxnotemax': 'Onyx Boox Note Max',
        'booxgo103': 'Onyx Boox Go 10.3'
    }
    
    pattern_names = {
        'dots': 'Dots',
        'lines': 'Lines',
        'grid': 'Grid',
        'blank': 'Blank'
    }
    
    for device in devices:
        for pattern in patterns:
            device_display = device_names[device]
            pattern_display = pattern_names[pattern]
            OUTPUT_FILE = f"{device_display} - {pattern_display}.pdf"
            
            print(f"\n{'='*60}")
            print(f"Generating {OUTPUT_FILE}...")
            print(f"{'='*60}")
            
            # Create the PDF
            create_dot_grid_pdf(
                filename=OUTPUT_FILE,
                spacing_mm=DOT_SPACING_MM,
                num_pages=NUM_PAGES,
                page_size=device,
                dot_radius=DOT_SIZE,
                page_number_position=PAGE_NUM_POS,
                left_margin_mm=LEFT_MARGIN,
                right_margin_mm=RIGHT_MARGIN,
                top_margin_mm=TOP_MARGIN,
                bottom_margin_mm=BOTTOM_MARGIN,
                include_title_page=INCLUDE_TITLE_PAGE,
                include_toc=INCLUDE_TOC,
                toc_line_spacing_mm=TOC_LINE_SPACING,
                page_pattern=pattern
            )
    
    print(f"\n{'='*60}")
    print(f"✓ Successfully generated {len(devices) * len(patterns)} PDF files!")
    print(f"{'='*60}")


Generating reMarkable 1 - Dots.pdf...
PDF created: reMarkable 1 - Dots.pdf
  - Content pages: 256
  - Title page: Yes
  - Table of Contents: Yes (12 page(s))
  - Total pages in PDF: 269
  - Page pattern: dots
  - Pattern spacing: 5mm
  - Page size: 157.0mm x 210.0mm
  - Margins: L=5mm, R=5mm, T=5mm, B=5mm
  - Page number position: lower-left
  - Font used: Arial
  - Serif font used: EBGaramond (embedded in PDF)

Generating reMarkable 1 - Lines.pdf...
PDF created: reMarkable 1 - Lines.pdf
  - Content pages: 256
  - Title page: Yes
  - Table of Contents: Yes (12 page(s))
  - Total pages in PDF: 269
  - Page pattern: lines
  - Pattern spacing: 5mm
  - Page size: 157.0mm x 210.0mm
  - Margins: L=5mm, R=5mm, T=5mm, B=5mm
  - Page number position: lower-left
  - Font used: Arial
  - Serif font used: EBGaramond (embedded in PDF)

Generating reMarkable 1 - Grid.pdf...
PDF created: reMarkable 1 - Grid.pdf
  - Content pages: 256
  - Title page: Yes
  - Table of Contents: Yes (12 page(s))
  - To